In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technial but must be specific so the customer understands the project(

Given a textfile that contains a message whether that be in a folder or not, we wish to be able to classify the said data into whether it is a spam email, or whether it is a legimate email through the usage of various machine learning models, though namely a multinomial naive bayes model in this case. 

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

The data was provided to me by Mr. Rivero within the mail folder as approximately 1000 separate text files. 

In [4]:
#uhruhughrughurhg
##data structure and common words
import os

#commenting this out cuz this is to clean my data c:
dir = 'mail-Copy1'
count = 0

for fileName in os.listdir(dir):
    f = os.path.join(dir, fileName)
#     if 'mail-Copy1/1spm' in str(f):
#         os.remove(f)

    count += 1

print(count, '\n')


874 



# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

Potential Bias: Sample size is too little to properly form conclusions, and it does not take into account proper demographics, only for that specific location, and in which case, this scenario would be a college. IT does not factor in the possibility that emails for different targets, for example mothers, or children would have different spam aimed at their specific demographic, in which for this example would be car insurance, or free game hacks. 

Furthermore, the inclusion of single string characters or single digit values may include some bias, and it may hold no significant value in the overall purpose, as well as containing similarities between the two datasets. For example, the letter 'a' or the number '0' may be used frequently, and can not be used to indicate neither spam nor nonspam. 

In addition, the sample data itself contains little to no logical sense, in which nonspam emails are sometimes to be considered (at least to the human) more nonsensical than spam. For example, a certain nonspam would be ". . . 's much restrictive s - > np np . 's " " np pro quite over-restriction , ."


In [2]:
##data structure and common words
import os

#commenting this out cuz this is to clean my data c:
dir = 'testData'
count = 0

for fileName in os.listdir(dir):
    f = os.path.join(dir, fileName)
#     if 'mail-Copy1/1spm' in str(f):
#         os.remove(f)

    count += 1

print('Count:', count, '\n')

import email
import email.policy

def get_email_structure(email):
    if isinstance(email, str):
        return email
    payload = email.get_payload()
    if isinstance(payload, list):
        return "multipart({})".format(", ".join([
            get_email_structure(sub_email)
            for sub_email in payload
        ]))
    else:
        return email.get_content_type()

from collections import Counter

def structures_counter(directory, dir):
    structures = Counter()
    for email in directory:
        filePlace = os.path.join(dir, email)
        # try:
        #     with open(filePlace, 'r', encoding='utf-8') as file:
        #         msg = file.read()
        #         structure = get_email_structure(msg)
        #         structures[structure] += 1
        # except IsADirectoryError:
        #     pass
        for email in emails:
            structure = get_email_structure(email)
            structures[structure] += 1
    return structures

#STINKY ALERT, DO NOT UNCOMMENT THIS, IT WILL BE VERY BAD
# print(structures_counter(os.listdir(dir), dir).most_common())

#------------------------------------------------------------

import glob
from collections import Counter
import re

def commonWords(text, n):
    # Define a regex pattern to match all punctuation characters
    punctuation = '[!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'
    #comment this in and out as needbe
    # nmb = '[0-9]+'
    
    punctuationNt = re.sub(punctuation, '', text)
    # nmbNt = re.sub(nmb, '', punctuationNt)
    # words = nmbNt.split()
    words = punctuationNt.split()
    pogWords = Counter(words)
    topWords = pogWords.most_common(n)
    return topWords

def processor(path, n):
    with open(path, 'rt') as file:
        return commonWords(file.read(), n)

def n1(dir, n):
    SPMcommonWords = Counter()
    nonSPMcommonWords = Counter()
    
    for filename in os.listdir(dir):
        if filename.endswith(".txt"):
            if filename[0] == '1':
                SPMpath = os.path.join(dir, filename)
                SPMtopWords = processor(SPMpath, n)
                SPMcommonWords.update(dict(SPMtopWords))
            else:
                nonSPMpath = os.path.join(dir, filename)
                nonSPMtopWords = processor(nonSPMpath, n)
                nonSPMcommonWords.update(dict(nonSPMtopWords))

    print(f"Top {n} most common words in the directory for SPM:")
    for word, count in SPMcommonWords.most_common(n):
        print(f"{word}: {count}")

    print('\n\n'+f"Top {n} most common words in the directory for nonSPM:")
    for word, count in nonSPMcommonWords.most_common(n):
        print(f"{word}: {count}")

n1('testData', 20)

Count: 874 

Top 20 most common words in the directory for SPM:
0: 1615
email: 1368
20: 1306
report: 1204
s: 1176
order: 1168
program: 834
address: 793
mail: 760
send: 758
our: 706
money: 617
name: 617
receive: 612
list: 594
3d: 579
business: 537
free: 498
com: 414
work: 410


Top 20 most common words in the directory for nonSPM:
language: 1400
university: 925
s: 713
de: 481
conference: 463
linguistic: 460
0: 457
1: 308
edu: 298
english: 274
30: 264
e: 237
information: 236
workshop: 223
1998: 220
one: 205
paper: 203
Subject: 196
2: 195
10: 190


# 4.Prepare the Data


Apply any data transformations and explain what and why

I did the following steps: 

I edited all my data, dropping all forms of punctuation. Things like "!" or "." don't serve much relevant purpose towards spam detection, as saying "This is SPAM!!!!" and "This is spam." convey the same meaning. 

After doing so, I renamed all my files to start with either 0 or 1 to convey whether it's non spam or spam. This is namely to help my model further along when I start comparing the data with the title in order to check if the predicted answer is correct. 

After doing so, I lemminated all text and saved it to a list. This is mainly done to make sure that when I do a word to vector transition in the next part, similar words with the same root are considered the same thing. For example, "I am very happy" vs "I am happier", happy and happier could be classified both as happy, in order to improve accuracy.

Finally I vectorized my words to an array, using the top 50 most common words as features. This is namely for future work with models, as they only can work with numbers. ------

In [1]:
#probably drop numbers and single letters because they aren't useful

import re
import os

#---------------------------------------------------------

#THE UH OH STINKY BUTTON
def uhOhStinky():
    cd = os.getcwd()
    for f in os.listdir(cd):
        if f.endswith(".txt"):
            z = os.path.join(cd, f)
            os.remove(z)

# uhOhStinky()

#stinky deleter 2
import shutil
# shutil.rmtree('testData')

#---------------------------------------------------------

#processor reset chucking this here for the future

#temp code, I'll probably fix the variables if I don't get lazy
def n2(dir):
    for n in os.listdir(dir): 
        z = os.path.join(dir, n)
        if z.endswith('.txt'):    
            with open(z, 'r') as f:
                q = f.read()
                punctuation = '[!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'
                qq = re.sub(punctuation, '', q)
                f.close()
    
            with open(z, 'w') as g:
                g.write(qq)
                g.close()

n2('testData')

print('Theoretically, hopefully, not broken done')

count = 0
for fileName in os.listdir('testData'):
    # f = os.path.join(dir, fileName)
#     if 'mail-Copy1/1spm' in str(f):
#         os.remove(f)
    count += 1

print('Count:', count, '\n')
        

Theoretically, hopefully, not broken done
Count: 874 



In [2]:
#processor part 2: Labeling data

#---------------------------------------------------------

#the stinky removinator:
#shutil.rmtree('testData2')

#---------------------------------------------------------
import os

def n3(dir):
    arr = ['0', '1']
    for n in os.listdir(dir):
        z = os.path.join(dir, n)
        if z.endswith('.txt') and n[0] not in arr:
            if 'spm' in z:
                os.rename(z, dir+'/1'+n)
            else: 
                os.rename(z, dir+'/0'+n)

n3('testData')

print('Poggies?? Done')

#---------------------------------------------------------

Poggies?? Done


In [5]:
#final transformation work i guess
#todo: 
#set it to a vector transformer, where the first 20 words are known
#also stem words ig

import numpy as np
import os
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib

#data stemming
#nltk.download('wordnet')
l = WordNetLemmatizer()

def n4(dir, q):
    if q.endswith('.txt'):
        with open(os.path.join(dir, q), 'r') as f:
            t = f.read()
            tL = nltk.word_tokenize(t)
            return ' '.join(tL)
            # return t

x = [n4('testData2', f) for f in os.listdir('testData2')]
y = [f[0] for f in os.listdir('testData2')] #where f[0] is 0 or 1

#print(x[6], '\n\n', y[6], '\n\n', len(x), len(y))

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(analyzer='word', max_features = 69)
x = cv.fit_transform([str(y) for y in x if y is not np.nan])

print(cv.get_feature_names_out(), '\n')
print(x.toarray())


#converting to a df
dF = pd.DataFrame({'value': [y[v] for v in range(len(y))],
                  'text': [x.toarray()[v] for v in range(len(y))]})

print(dF.head())
print(dF.tail())

xTr, XTe, YTr, YTe = train_test_split(x, y, test_size=0.2, random_state=42)

joblib.dump(cv, 'cv.pkl')

['10' '100' '20' '30' '3d' '50' 'address' 'business' 'call' 'check' 'com'
 'conference' 'day' 'de' 'each' 'edu' 'email' 'english' 'even' 'fax'
 'first' 'follow' 'form' 'free' 'here' 'http' 'include' 'information'
 'interest' 'internet' 'language' 'letter' 'linguistic' 'list' 'mail'
 'many' 'market' 'money' 'most' 'name' 'need' 'nt' 'number' 'offer' 'one'
 'order' 'our' 'over' 'paper' 'please' 'product' 'program' 'read'
 'receive' 'report' 'send' 'service' 'site' 'start' 'subject' 'system'
 'those' 'university' 'us' 'want' 'web' 'week' 'work' 'www'] 

[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [1 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]]
  value                                               text
0     0  [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 5, 0, ...
1     0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2     0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3     0  [0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 1, 0, 0, 5, 1, ...
4     0  [0, 0, 0, 0, 0, 0, 

['cv.pkl']

In [55]:
#it would be really funny to have a pipeline
#leaving this here, not to do anyting

# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with

I used multinomial naive bayes, as that is one of the most popular models to use in regards to natural language processing. Seeing that I got a 96% accuracy, I see it fit to move on to fine tuning and seeing what best would aid my work. 

In [9]:
from sklearn.naive_bayes import MultinomialNB
import joblib

m = MultinomialNB()
m.fit(xTr, YTr)
print("Accuracy: ", m.score(XTe, YTe)*100, '%', sep='')

joblib.dump(m, "m.pkl")

Accuracy: 96.0%


['m.pkl']

# 6. Fine Tune the Model

With the select model descibe the steps taken to acheve the best rusults possiable 

I gridsearched the results using the possible parameters without breaking my model. I used cv=5 because that'd let me 5fold score, giving me the best possible result. The only issue that occured is that MultinomialNB may only contain 1 parameter, which is alpha. It doesn't do much, making it largely useless and the gridsearch irrelevant


In [10]:
from sklearn.model_selection import GridSearchCV

param_grid = {'alpha': [0.1, 0.5, 1.0, 2.0]}
grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=5)
grid_search.fit(xTr, YTr)
best_alpha = grid_search.best_params_['alpha']
print(best_alpha)

print('Done')

0.1
Done


/home/jupyter-michaelw/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


# 7. Present
In a customer faceing Document provide summery of finding and detail approach taken


1. Upon Recieving the Data, it was immediately converted to a readable format for the machine. This was largely done by first removing all punctuation, labeling the data, and finally, converting the words to a readable format (namely numbers) for the machine.  
2. The model was then trained using a Multinomial Naive Bayes model, which turned out to be quite successful. This is largely due to the relatively obvious split in the data as demonstrated in Step 3 where we looked for patterns.
3. Upon gridsearching it --a method in which we try to find the best possible parameters to improve our model--, we were unable to significantly improve it, leaving the accuracy at 96%. If we were to improve, we'd most likely try to gather more data, as gridsearching is likely to not improve things due to the singular parameter. 
   

# 8. Launch the Model System
Define your production run code, This should be self susficent and require only your model pramaters 


In [ ]:
def n7(*args):
    import os, re, joblib, nltk
    import numpy as np

    x = []
    
    def n2(dir, z): 
        if os.path.isdir(dir):
            z = os.path.join(dir, z)
            
        if z.endswith('.txt'):
            with open(z, 'r') as f:
                q = f.read()
                punctuation = '[!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'
                qq = re.sub(punctuation, '', q)
                f.close()
        
            with open(z, 'w') as g:
                g.write(qq)
                g.close()

    def n4(dir, z):
        if os.path.isdir(dir):
            z = os.path.join(dir, z)
            
        if z.endswith('.txt'):
            with open(z, 'r') as f:
                t = f.read()
                tL = nltk.word_tokenize(t)
                return ' '.join(tL)

    def n9(x):
        cv = joblib.load('cv.pkl')
        m = joblib.load('m.pkl')

        x = cv.fit_transform([str(y) for y in x if y is not np.nan])
        
        c, l = 0, 10
        for y in x:
            pV = m.predict(y)

            if pV[0] == '1':
                aN = 'Spam'
            else:
                aN = 'Non-Spam'
                
            if c != -1:
                c+= 1
                
            if c <= l:
                print('{}. Predicted Value for File {}:'. format(c, c), aN)
            else:
                u = str(input('\nPrint Limit Hit. Enter Y to display all Results, and C to display the next 10 Predictions')).lower()
                print('\n')

                if u == 'y':
                    c = -1
                elif u == 'c':
                    l += 10
                else: 
                    return 0
    
    
    #we're going to assume that our arguement is a file path
    for n in args:
        n = str(n)
        if n.endswith('.txt') == False:
            d = False
            if os.path.isdir(n) == False:
                print('Function Execution Failed. Try Again')
                return 0
            else:
                d = True
                
        if d:
            for p in os.listdir(n):
                n2(n, p)
                x.append([n4(n, p)])
        else: 
            n2(n, n)
            x.append([n4(n, n)])

    n9(x)
            
n7('wuhwuh')
print('Done')

1. Predicted Value for File 1: Non-Spam
2. Predicted Value for File 2: Non-Spam
3. Predicted Value for File 3: Non-Spam
4. Predicted Value for File 4: Non-Spam
5. Predicted Value for File 5: Non-Spam
6. Predicted Value for File 6: Non-Spam
7. Predicted Value for File 7: Spam
8. Predicted Value for File 8: Non-Spam
9. Predicted Value for File 9: Non-Spam
10. Predicted Value for File 10: Non-Spam



Print Limit Hit. Enter Y to display all Results, and C to display the next 10 Predictions c




12. Predicted Value for File 12: Non-Spam
13. Predicted Value for File 13: Non-Spam
14. Predicted Value for File 14: Spam
15. Predicted Value for File 15: Spam
16. Predicted Value for File 16: Non-Spam
17. Predicted Value for File 17: Non-Spam
18. Predicted Value for File 18: Non-Spam
19. Predicted Value for File 19: Non-Spam
20. Predicted Value for File 20: Spam
